In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
from random import randint

In [2]:
# Define a dictionary headers to store the User-Agent string for the request
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [3]:
# Read the player links from the csv file
df_links = pd.read_csv('output/player_links.csv')

# Convert the 'URL' column of the dataframe to a list
links = df_links['URL'].tolist()

In [7]:
# List to store data of each player in the form of a dictionary
list_of_row_dicts = []

# Loop through each link to scrape player data
for link in links:

    # Keep trying until the request is successful
    while True:
        try:
            # Make a GET request to the link
            request = rq.get(link,headers=headers)
            
            # Extraire les morceaux utiles
            parts = link.split('/')
            player_slug = parts[3]
            player_id = parts[-1]
            
            # Construire l'URL de stats détaillées Ligue 1 saison 2024
            competition = "FR1"
            season = "2024"        
            detailed_url = f"https://www.transfermarkt.com/{player_slug}/leistungsdatendetails/spieler/{player_id}/wettbewerb/{competition}/saison/{season}"
            request_detailed = rq.get(detailed_url, headers=headers)
        
            # Use BeautifulSoup to parse the HTML content of the page
            soup = bs(request.text, 'html.parser')
            soup_detailed = bs(request_detailed.text, 'html.parser')

            # Find all <span> tags in the HTML
            title_spans = soup.find_all('span')

            table = soup_detailed.find('table', class_='items')
            print(table)
                       
            # Break out of the loop if the request is successful
            break
        except AttributeError:
            # Print error message and wait for 10 seconds before retrying
            print('Index Error : Sleeping for 10 seconds before retrying')
            sleep(10)
            
    try:
        name = ' '.join([word for word in soup.find('h1').text.split() if not any(i.isdigit() for i in word)]).strip()
    except IndexError:
        name = None
        print ('Name not found')

    try:
        date_of_birth = [span.find_next('span').text for span in title_spans if 'Date of birth/Age:' in span.text][0].strip()
    except IndexError:
        date_of_birth = None
        print ('DoB not found')

    try:
        citizenship = [span.find_next('span').text.strip().split('\xa0\xa0') for span in title_spans if 'Citizenship:' in span.text][0]
    except AttributeError:
        citizenship = None
        print ('Citizenship not found')

    try:
        current_club = [span.find_next('span').text for span in title_spans if 'Current club:' in span.text][0].strip()
    except IndexError:
        current_club = None
        print ('Current club not found')

    try:
        outfitter = [span.find_next('span').text for span in title_spans if 'Outfitter:' in span.text][0].strip()
    except IndexError:
        outfitter = None
        print ('Outfitter not found')

    try:
        mv = [span.next_sibling.strip() for span in title_spans if '€' in span.text][0].strip()
        korm = [span.find_next('span').text for span in title_spans if '€' in span.text][0].strip()
        mv = mv + korm
    except IndexError:
        mv = None
        print ('MV not found')

    try:
        # Essayer d'obtenir l'entête via thead
        header_row = table.find("thead")
        if header_row:
            headers = header_row.find_all("th")
        else:
            # Si pas de thead, on prend le premier tr du tbody
            first_row = table.find("tbody").find("tr")
            headers = first_row.find_all("td")  # souvent les entêtes sont stylées comme des td
        
        # Identifier l’index de la colonne "Goals"
        goal_index = None
        for i, h in enumerate(headers):
            if "Goal" in h.get_text():
                goal_index = i
                break
        
        # Prendre la première ligne de stats réelle (evite la ligne d'entête si on a dû la prendre du tbody)
        rows = table.find("tbody").find_all("tr", class_=["odd", "even"])
        stats_row = rows[0] if rows else None
        
        if goal_index is not None and stats_row:
            cells = stats_row.find_all("td")
            if goal_index < len(cells):
                goals = cells[goal_index].text.strip()
                print("Buts marqués :", goals)
            else:
                print("Index de but hors limites.")
        else:
            print("Impossible de trouver la colonne des buts ou ligne de stats.")
    except IndexError:
        goals = None
        print('Stats not found')
            
    row_dic = {
    'PLAYER_URL' : link,
    'NAME' : name,
    'DATE_OF_BIRTH' : date_of_birth,
    'CITIZENSHIP': citizenship,
    'CURRENT_CLUB' : current_club,
    'OUTFITTER' : outfitter,
    'MV' : mv,
    'MATCHS' : matchs,
    'GOALS' : goals,
    'ASSISTS' : assists
    }
    list_of_row_dicts.append(row_dic)
            
    print(name,',',date_of_birth,',',citizenship,',',current_club,',',outfitter,',',mv,',',matchs,',',goals,',',assists)
    sleep(randint(1,3))

df = pd.DataFrame(list_of_row_dicts)

Index Error : Sleeping for 10 seconds before retrying
Index Error : Sleeping for 10 seconds before retrying


KeyboardInterrupt: 

In [132]:
df.to_csv('output/player_info_details.csv')